In [127]:
import scipy
import numpy as np
from scipy.stats import unitary_group
import itertools

In [128]:
U = unitary_group.rvs(6)

type(U)

numpy.ndarray

In [129]:
def get_alpha(index: list, unitary: np.ndarray):

    input_mode_occupations, output_mode_occupations = index[0:3] + [1,1,1], index[3:6] + [1,1,1]

    n_input = sum(input_mode_occupations)
    n_output = sum(output_mode_occupations)

    if n_input != n_output:

        return 0

    occupied_input_modes = [index for index, occupation in enumerate(input_mode_occupations) if occupation==1]

    idk_how_to_name_this = []
    for mode, occupation in enumerate(output_mode_occupations):
        for _ in range(occupation):
            idk_how_to_name_this.append(mode)

    permutations = list(itertools.permutations(idk_how_to_name_this))

    alpha = 0

    for permutation in permutations:
        poly = 1
        for index, mode in enumerate(occupied_input_modes):
            poly *= unitary[mode, permutation[index]]

        alpha += poly
    
    return alpha

In [135]:
#def partition_min_max(n, k, l, m):
#    """
#    n: The integer to partition
#    k: The length of partitions
#    l: The minimum partition element size
#    m: The maximum partition element size
#    """
#    if k < 1:
#        return
#    if k == 1:
#        if l <= n <= m:
#            yield (n,)
#        return
#    for i in range(l, m + 1):
#        for result in partition_min_max(n - i, k - 1, i, m):
#            yield result + (i,)

def partition_min_max(n, k, l, m):
    """
    n: The integer to partition
    k: The length of partitions
    l: The minimum partition element size
    m: The maximum partition element size
    """
    if k < 1:
        return []
    if k == 1:
        if l <= n <= m:
            return [(n,)]
        return []
    result = []
    for i in range(l, m + 1):
        sub_partitions = partition_min_max(n - i, k - 1, i, m)
        for sub_partition in sub_partitions:
            result.append(sub_partition + (i,))
    return result

def get_partitions_permutations(n, k):
    partitions = partition_min_max(n, k, 0, n)

    # print(partitions)

    permutations = []
    for partition in partitions:
        permutations.extend(list(itertools.permutations(partition)))
    
    return list(map(list, list(set(permutations))))
            
# Example usage:
N = 3
M = 3
# partitions = list(partition_min_max(N, M, 0, N))
permutations = get_partitions_permutations(N, M)
# print(partitions)
print(permutations)

[[0, 2, 1], [0, 0, 3], [3, 0, 0], [2, 1, 0], [1, 2, 0], [0, 3, 0], [2, 0, 1], [0, 1, 2], [1, 0, 2], [1, 1, 1]]


In [136]:
def loss_function_ccz_dual_rail(U):
    desired_gate_loss = 0
    + np.abs(get_alpha([0,0,0,0,0,0], U) - get_alpha([0,1,1,0,1,1], U))**2 
    + np.abs(get_alpha([0,1,1,0,1,1], U) - get_alpha([1,0,1,1,0,1], U))**2
    + np.abs(get_alpha([1,0,1,1,0,1], U) - get_alpha([1,1,0,1,1,0], U))**2
    + np.abs(get_alpha([1,1,0,1,1,0], U) + get_alpha([0,0,1,0,0,1], U))**2
    + np.abs(get_alpha([0,0,1,0,0,1], U) - get_alpha([0,1,0,0,1,0], U))**2
    + np.abs(get_alpha([0,1,0,0,1,0], U) - get_alpha([1,0,0,1,0,0], U))**2
    + np.abs(get_alpha([1,0,0,1,0,0], U) - get_alpha([1,1,1,1,1,1], U))**2

    undesired_gate_loss = 0

    for input_state in [[0,0,0], [0,0,1], [0,1,0], [0,1,1], [1,0,0], [1,0,1], [1,1,0], [1,1,1]]:
        
        particle_number = np.sum(input_state)

        output_states = get_partitions_permutations(n=particle_number, k=3)


        for output_state in output_states:
            if input_state != output_state:
                
                index=input_state + list(output_state) 
                undesired_gate_loss += np.abs(get_alpha(index=index, unitary=U))**2

    loss = desired_gate_loss + undesired_gate_loss

    return loss

In [140]:
def get_success_prob(U):
    return np.abs(get_alpha(index=[0,0,0,0,0,0], unitary=U))**2

In [144]:
best_loss = np.infty
best_prob = 0
best_U = None

for _ in range(3000):

    U = unitary_group.rvs(6)

    loss = loss_function_ccz_dual_rail(U=U)
    prob = get_success_prob(U)

    if loss < best_loss:
        best_loss = loss
        best_prob = prob
        best_U = U

#print(loss_function_ccz_dual_rail(U=U))
#print(get_success_prob(U))

In [145]:
print(best_loss, best_prob)
print(U)

0.07625325382063168 0.004201985572322366
[[ 0.35386082+0.09370338j -0.11781769+0.1126784j  -0.59085513+0.12103865j
  -0.24731624-0.22348081j  0.34500912-0.437993j    0.23122927+0.01481521j]
 [ 0.05147172-0.34501342j -0.61937862-0.20981538j  0.15924725-0.19038506j
   0.40548284+0.04569049j -0.04232555-0.28167285j  0.36699666-0.08207511j]
 [-0.00536796-0.40540113j -0.1896524 -0.03166948j -0.47094997-0.07977378j
  -0.22194236+0.47062341j  0.08917585+0.51562436j  0.00107132-0.16101437j]
 [ 0.2145497 -0.32452371j -0.33890926-0.06335179j  0.16423531+0.05956836j
  -0.16663138-0.28107991j  0.1844601 -0.02745294j -0.73554127+0.12917484j]
 [ 0.52238862+0.32440679j -0.32911184+0.50325446j  0.21671374+0.18323404j
   0.11193591+0.20973073j -0.04669289+0.30659866j  0.07758255+0.1450259j ]
 [ 0.21748837-0.05622874j  0.16668428-0.02372708j -0.47853037-0.08014661j
   0.53096638+0.08017722j -0.43980321-0.09894416j -0.30082001+0.32204433j]]


In [24]:
permutations = list(itertools.permutations([1, 2, 3]))
print(permutations)

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


In [55]:
from scipy.optimize import minimize
import numpy as np
import scipy.linalg
from scipy.stats import unitary_group

N = 6
initial_guess = unitary_group.rvs(N)
mode_set = [4, 5, 6]

def objective_function(M: np.ndarray):
    function = 1
    for r in range(len(mode_set)):
        function *= sum(M[r])
    return function

def unitary_constraint(M: np.ndarray):
    return scipy.linalg.norm(M - np.eye(N))  # || M - I ||

constraint = {'type': 'eq', 'fun': unitary_constraint}

def real_to_complex(z):  # real vector of length 2n -> complex of length n
    return z[:len(z) // 2] + 1j * z[len(z) // 2:]

def complex_to_real(z):  # complex vector of length n -> real of length 2n
    return np.concatenate((np.real(z), np.imag(z)))

# Convert the generator expression into a list
initial_guess_flat = complex_to_real(initial_guess.flatten())

result = minimize(
    lambda z: objective_function(real_to_complex(z)),
    initial_guess_flat,
    constraints=constraint
)

print("Optimized Unitary Matrix:")
print(result.x.reshape((N, N)))


ValueError: operands could not be broadcast together with shapes (72,) (6,6) 